In [2]:
from imports.ASLDataset import ASLDataset
import torch
#torch.set_default_dtype(torch.float32) 

l = ASLDataset(letters=['a', 'b'], overwrite_processing=False)

Processing...


processing:   0%|          | 0/8400 [00:00<?, ?it/s]

Done!


In [3]:
from imports.NVSModel import ResGNet, og_Net, Net

model = Net(2).to('cuda')
model

Net(
  (conv_blocks): ModuleDict(
    (conv_block1): ConvBlock(
      (conv): SplineConv(1, 64, dim=3)
      (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_block2): ConvBlock(
      (conv): SplineConv(64, 128, dim=3)
      (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_block3): ConvBlock(
      (conv): SplineConv(128, 256, dim=3)
      (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_block4): ConvBlock(
      (conv): SplineConv(256, 512, dim=3)
      (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (pool_blocks): ModuleDict(
    (pool_block1): MaxPooling(voxel_size=20)
    (pool_block2): MaxPooling(voxel_size=30)
    (pool_block3): MaxPooling(voxel_size=50)
    (pool_block4): MaxPoolingX(voxel_size=80, size=64)
  )
  (fc1): Linear(in_features=32768, out_features=1024, bias=T

In [4]:
from torch_geometric.loader import DataLoader

dl = DataLoader(l, 3, shuffle=True)

In [5]:
batch = next(iter(dl)).to('cuda')

In [6]:
model(batch)

tensor([[-0.7028, -0.6836],
        [-0.6094, -0.7846],
        [-0.6874, -0.6989]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

In [24]:
from torchmetrics import Accuracy

Acc=Accuracy()


In [16]:
batch.y

tensor([1, 1, 0], device='cuda:0')

In [33]:
import torch.nn.functional as F


def train(epoch, train_loader):
    model.train()

    if epoch == 60:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001

    if epoch == 110:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.00001

    for i, data in enumerate(tqdm(train_loader, desc='batches')):
        with torch.autograd.detect_anomaly():
            data = data.to('cuda')
            #print(data.y)
            optimizer.zero_grad()
            end_point = model(data)
            
            loss = F.nll_loss(end_point, data.y)
            pred = end_point.max(1)[1]
            
            acc = (pred.eq(data.y).sum().item())/len(data.y)
            
            loss.backward()
            optimizer.step()
            
            if i % 5 == 0:
                print(data.y, pred, Acc(data.y.to('cpu'), pred.to('cpu')))
                print({'epoch': epoch,'batch': i + 1,'loss': loss.item(),'acc': acc})
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [29]:
"" 

''

In [34]:
model.train()

Net(
  (conv_blocks): ModuleDict(
    (conv_block1): ConvBlock(
      (conv): SplineConv(1, 64, dim=3)
      (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_block2): ConvBlock(
      (conv): SplineConv(64, 128, dim=3)
      (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_block3): ConvBlock(
      (conv): SplineConv(128, 256, dim=3)
      (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_block4): ConvBlock(
      (conv): SplineConv(256, 512, dim=3)
      (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (pool_blocks): ModuleDict(
    (pool_block1): MaxPooling(voxel_size=20)
    (pool_block2): MaxPooling(voxel_size=30)
    (pool_block3): MaxPooling(voxel_size=50)
    (pool_block4): MaxPoolingX(voxel_size=80, size=64)
  )
  (fc1): Linear(in_features=32768, out_features=1024, bias=T

In [ ]:
from tqdm.auto import tqdm


for epoch in range(1, 2):
    
    #train_data_aug = T.Compose([T.Cartesian(cat=False), T.RandomFlip(axis=0, p=0.3), T.RandomScale([0.95,0.999]), T.RandomFlip(axis=1, p=0.2)])
    train_loader = dl
    
    train(epoch, train_loader)

batches:   0%|          | 0/2800 [00:00<?, ?it/s]

/tmp/ipykernel_14667/634645094.py:16: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():


tensor([0, 1, 0], device='cuda:0') tensor([0, 1, 0], device='cuda:0') tensor(1.)
{'epoch': 1, 'batch': 1, 'loss': 0.23057891428470612, 'acc': 1.0}
tensor([0, 0, 1], device='cuda:0') tensor([1, 1, 0], device='cuda:0') tensor(0.)
{'epoch': 1, 'batch': 6, 'loss': 6.120129108428955, 'acc': 0.0}
tensor([0, 1, 1], device='cuda:0') tensor([0, 1, 0], device='cuda:0') tensor(0.6667)
{'epoch': 1, 'batch': 11, 'loss': 0.28697383403778076, 'acc': 0.6666666666666666}
tensor([1, 1, 1], device='cuda:0') tensor([1, 1, 0], device='cuda:0') tensor(0.6667)
{'epoch': 1, 'batch': 16, 'loss': 0.23637212812900543, 'acc': 0.6666666666666666}
tensor([0, 1, 0], device='cuda:0') tensor([1, 1, 0], device='cuda:0') tensor(0.6667)
{'epoch': 1, 'batch': 21, 'loss': 6.32385778427124, 'acc': 0.6666666666666666}
tensor([0, 0, 1], device='cuda:0') tensor([0, 1, 0], device='cuda:0') tensor(0.3333)
{'epoch': 1, 'batch': 26, 'loss': 2.0908076763153076, 'acc': 0.3333333333333333}
tensor([1, 0, 0], device='cuda:0') tensor([1

In [ ]:
import torch
model = torch.load('model.pkl')
model

In [ ]:
import pandas as pd

log = pd.read_csv('log', names = ['epoch', 'batch', 'loss', 'acc'])
log

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


plt.plot(log['loss'])

In [ ]:
sns.lineplot(x=log.index, y=log['loss'], )

In [ ]:
from imports.NVSModel import Net

model = Net(
    n_classes=2
    #layer_sizes=[64, 128],
    #voxel_sizes=[4, 6]
)
model

In [ ]:
model.fc1

In [ ]:
512*64